In [1]:
import MetaTrader5 as mt5

In [2]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

ticker = 'RLXm'
info = mt5.symbol_info(ticker)

In [3]:
def transformar_a_uno(numero):
    # Inicializar contador de decimales
    decimales = 0
    while numero != int(numero):
        numero *= 10
        decimales += 1
    # Retornar 1 dividido por 10 elevado a la cantidad de decimales
    return 1 / (10 ** decimales)

In [21]:
contract_volume = info.trade_contract_size
minimum_lot = info.volume_min
maximum_lot = info.volume_max

pip_value = info.point
minimum_units = contract_volume * minimum_lot
maximum_units = contract_volume * maximum_lot

print('contract_volume', contract_volume)
print('minimum_lot', minimum_lot)
print('maximum_lot', maximum_lot)
print('minimum_units: ', minimum_units)
print('maximum_units: ', maximum_units)
print('pip_value: ', pip_value)

minimum_fraction = transformar_a_uno(minimum_units)
print('minimum_fraction: ', minimum_fraction)


scaled_contract_volume = contract_volume / minimum_fraction


price = info.ask
scaled_price = price * minimum_fraction
scaled_minimum_units = minimum_lot * scaled_contract_volume
scaled_pip_value = pip_value * minimum_fraction

print('original_price: ', price)
print('scaled_price:', scaled_price)
print('scaled_minimum_units:', scaled_minimum_units)
print('scaled_pip_value:', scaled_pip_value)

contract_volume 100.0
minimum_lot 0.01
maximum_lot 10.0
minimum_units:  1.0
maximum_units:  1000.0
pip_value:  0.01
minimum_fraction:  1.0
original_price:  1.67
scaled_price: 1.67
scaled_minimum_units: 1.0
scaled_pip_value: 0.01


In [5]:
def diff_pips(price1, price2, pip_value, absolute=True):
    if absolute:
        difference = abs(price1 - price2)
    else:
        difference = price1 - price2
    pips = difference / pip_value
    
    return pips

In [6]:
diff_pips(42417.7, 42411.7, pip_value=pip_value)

600.0

In [7]:
diff_pips(424.177, 424.117, pip_value=pip_value)

6.000000000000227

In [8]:
diff_pips(424.177, 424.117, pip_value=scaled_pip_value)

6.000000000000227

# Experimento

In [9]:
import pytz
from datetime import datetime
import pandas as pd


timezone = pytz.timezone("Etc/UTC")

if not mt5.initialize():
    raise Exception("initialize() failed, error code =", mt5.last_error())

date_from_get_data = datetime(2021, 10, 1, tzinfo=timezone)
date_to_get_data = datetime(2024, 9, 1, tzinfo=timezone)

# Obtener las tasas históricas
rates = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_H4, date_from_get_data, date_to_get_data)

# Crear DataFrame con las tasas
df = pd.DataFrame(rates)

# Convertir el tiempo de segundos a formato datetime
df['time'] = pd.to_datetime(df['time'], unit='s')

# Renombrar columnas para el ticker principal
df = df.rename(columns={
    'time': 'Date',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'tick_volume': 'Volume'
}).set_index('Date')

df.index = df.index.tz_localize('UTC').tz_convert('UTC')

mt5.shutdown()

df

,Open,High,Low,Close,Volume,spread,real_volume
Date,,,,,,,
2021-10-27 16:00:00+00:00,5.03,5.05,5.03,5.05,3,7,0
2021-10-28 12:00:00+00:00,4.92,5.04,4.87,4.91,95,7,0
2021-10-28 16:00:00+00:00,4.92,4.94,4.89,4.91,44,7,0
2021-10-29 12:00:00+00:00,4.97,5.00,4.83,4.93,101,7,0
2021-10-29 16:00:00+00:00,4.92,4.92,4.80,4.84,23,7,0
...,...,...,...,...,...,...,...
2024-08-28 16:00:00+00:00,1.61,1.61,1.60,1.60,130,4,0
2024-08-29 12:00:00+00:00,1.62,1.66,1.62,1.66,116,4,0
2024-08-29 16:00:00+00:00,1.66,1.67,1.66,1.66,150,4,0


In [10]:
from backbone.utils.general_purpose import calculate_units_size
from backtesting import Strategy, Backtest
import talib as ta

class BPercent(Strategy):
    pip_value = None
    minimum_units = None
        
    risk= 1
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200
    b_open_threshold = 0.90
    b_close_threshold = 0.5
    atr_multiplier = 1.8
    pip_value = 0.1

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )
        
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)
        

    def next(self):
        actual_close = self.data.Close[-1]
        b_percent = (actual_close - self.lower_band[-1]) / (self.upper_band[-1] - self.lower_band[-1])
        
        if self.position:
            if self.position.is_long:
                if b_percent >= self.b_close_threshold:
                    self.position.close()

            if self.position.is_short:
                if b_percent <= 1 - self.b_close_threshold:
                    self.position.close()

        else:
            if b_percent <= 1 - self.b_open_threshold and actual_close > self.sma[-1]:

                sl_price = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                units = self.minimum_units if units < self.minimum_units else units
                
                self.buy(
                    size=units,
                    # sl=sl_price
                )
                
            if b_percent >= self.b_open_threshold and actual_close < self.sma[-1]:
                sl_price = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                units = self.minimum_units if units < self.minimum_units else units

                self.sell(
                    size=units,
                    # sl=sl_price    
                )

In [11]:
bt = Backtest(df, BPercent, commission=7e-4, margin=1/30, cash=15_000)

stats = bt.run(
    pip_value = pip_value,
    minimum_units = 10
)

bt.plot(filename='./original.html')

stats

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2021-10-27 16:00...
End                       2024-08-30 16:00...
Duration                   1038 days 00:00:00
Exposure Time [%]                   29.141836
Equity Final [$]                 14629.341516
Equity Peak [$]                     15744.418
Return [%]                          -2.471057
Buy & Hold Return [%]              -67.722772
Return (Ann.) [%]                   -0.877982
Volatility (Ann.) [%]                6.052033
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -11.810662
Avg. Drawdown [%]                   -2.009623
Max. Drawdown Duration      669 days 00:00:00
Avg. Drawdown Duration       92 days 16:00:00
# Trades                                   14
Win Rate [%]                        57.142857
Best Trade [%]                      15.600261
Worst Trade [%]                    -67.974725
Avg. Trade [%]                    

In [12]:
scaled_prices = df[['Open','High','Low','Close']] * minimum_fraction
scaled_prices

,Open,High,Low,Close
2021-10-27 16:00:00+00:00,5.03,5.05,5.03,5.05
2021-10-28 12:00:00+00:00,4.92,5.04,4.87,4.91
2021-10-28 16:00:00+00:00,4.92,4.94,4.89,4.91
2021-10-29 12:00:00+00:00,4.97,5.00,4.83,4.93
2021-10-29 16:00:00+00:00,4.92,4.92,4.80,4.84
...,...,...,...,...
2024-08-28 16:00:00+00:00,1.61,1.61,1.60,1.60
2024-08-29 12:00:00+00:00,1.62,1.66,1.62,1.66
2024-08-29 16:00:00+00:00,1.66,1.67,1.66,1.66
2024-08-30 12:00:00+00:00,1.64,1.65,1.63,1.64


In [13]:
bt = Backtest(scaled_prices, BPercent, commission=7e-4, margin=1/30, cash=1_000)

stats = bt.run(
    pip_value = scaled_pip_value,
    minimum_units = 10
)

bt.plot(filename='./escalado.html')
stats

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2021-10-27 16:00...
End                       2024-08-30 16:00...
Duration                   1038 days 00:00:00
Exposure Time [%]                   29.141836
Equity Final [$]                    975.77746
Equity Peak [$]                   1049.483066
Return [%]                          -2.422254
Buy & Hold Return [%]              -67.722772
Return (Ann.) [%]                   -0.860503
Volatility (Ann.) [%]                6.031946
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -11.742924
Avg. Drawdown [%]                   -2.001069
Max. Drawdown Duration      669 days 00:00:00
Avg. Drawdown Duration       92 days 16:00:00
# Trades                                   14
Win Rate [%]                        57.142857
Best Trade [%]                      15.600261
Worst Trade [%]                    -67.974725
Avg. Trade [%]                    

In [14]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-39,206,227,3.637452,3.07,22.130628,0.156003,2022-02-10 16:00:00+00:00,2022-02-22 16:00:00+00:00,12 days 00:00:00
1,-54,332,343,2.018586,1.83,10.183644,0.093425,2022-04-28 12:00:00+00:00,2022-05-05 16:00:00+00:00,7 days 04:00:00
2,-53,357,365,2.038572,1.85,9.994316,0.092502,2022-05-16 16:00:00+00:00,2022-05-20 16:00:00+00:00,4 days 00:00:00
3,-62,385,420,2.028579,2.09,-3.808102,-0.030278,2022-06-06 16:00:00+00:00,2022-07-01 12:00:00+00:00,24 days 20:00:00
4,-79,583,685,1.399020,2.35,-75.127420,-0.679747,2022-10-26 16:00:00+00:00,2022-12-19 16:00:00+00:00,54 days 00:00:00
5,62,703,715,2.331631,2.67,20.978878,0.145121,2022-12-28 16:00:00+00:00,2023-01-04 16:00:00+00:00,7 days 00:00:00
6,71,766,829,2.441708,2.19,-17.871268,-0.103087,2023-01-30 16:00:00+00:00,2023-03-01 16:00:00+00:00,30 days 00:00:00
7,-61,850,871,2.368341,2.33,2.338801,0.016189,2023-03-10 16:00:00+00:00,2023-03-24 16:00:00+00:00,14 days 00:00:00
8,-133,1125,1138,1.548915,1.50,6.505695,0.031580,2023-09-26 16:00:00+00:00,2023-10-05 12:00:00+00:00,8 days 20:00:00
9,-150,1143,1195,1.568901,1.60,-4.664850,-0.019822,2023-10-09 16:00:00+00:00,2023-11-10 16:00:00+00:00,32 days 00:00:00


In [15]:
scaled_contract_volume

100.0

In [16]:
trades = stats._trades.copy()

trades.Size = trades.Size / scaled_contract_volume
trades.EntryPrice = trades.EntryPrice / minimum_fraction
trades.ExitPrice = trades.ExitPrice / minimum_fraction

trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-0.39,206,227,3.637452,3.07,22.130628,0.156003,2022-02-10 16:00:00+00:00,2022-02-22 16:00:00+00:00,12 days 00:00:00
1,-0.54,332,343,2.018586,1.83,10.183644,0.093425,2022-04-28 12:00:00+00:00,2022-05-05 16:00:00+00:00,7 days 04:00:00
2,-0.53,357,365,2.038572,1.85,9.994316,0.092502,2022-05-16 16:00:00+00:00,2022-05-20 16:00:00+00:00,4 days 00:00:00
3,-0.62,385,420,2.028579,2.09,-3.808102,-0.030278,2022-06-06 16:00:00+00:00,2022-07-01 12:00:00+00:00,24 days 20:00:00
4,-0.79,583,685,1.399020,2.35,-75.127420,-0.679747,2022-10-26 16:00:00+00:00,2022-12-19 16:00:00+00:00,54 days 00:00:00
5,0.62,703,715,2.331631,2.67,20.978878,0.145121,2022-12-28 16:00:00+00:00,2023-01-04 16:00:00+00:00,7 days 00:00:00
6,0.71,766,829,2.441708,2.19,-17.871268,-0.103087,2023-01-30 16:00:00+00:00,2023-03-01 16:00:00+00:00,30 days 00:00:00
7,-0.61,850,871,2.368341,2.33,2.338801,0.016189,2023-03-10 16:00:00+00:00,2023-03-24 16:00:00+00:00,14 days 00:00:00
8,-1.33,1125,1138,1.548915,1.50,6.505695,0.031580,2023-09-26 16:00:00+00:00,2023-10-05 12:00:00+00:00,8 days 20:00:00
9,-1.50,1143,1195,1.568901,1.60,-4.664850,-0.019822,2023-10-09 16:00:00+00:00,2023-11-10 16:00:00+00:00,32 days 00:00:00
